In [1]:
pip install sentence_transformers datasets evaluate rouge-score nltk transformers[torch]==4.35.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for transformers[torch]==4.35.0 from https://files.pythonhosted.org/packages/9a/06/e4ec2a321e57c03b7e9345d709d554a52c33760e5015fdff0919d9459af0/transformers-4.35.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 5.6 MB/s eta 0:00:00
  Obtaining dependency information for tokenizers<0.15,>=0.14 from https://files.pythonhosted.org/packages/a7/7b/c1f643eb086b6c5c33eef0c3752e37624bd23e4cbc9f1332748f1c6252d1/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for huggingface-hub<1.0,>=0.16.4 fro

In [2]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import os
import itertools
import spacy
import nltk
from nltk.tokenize import sent_tokenize
from datasets import load_metric
from tqdm import tqdm
import transformers
from transformers import AutoTokenizer

print(transformers.__version__)

pd.set_option('display.max_colwidth', None)
nltk.download('punkt')
nlp = spacy.load("en_core_web_sm")
rouge = load_metric("rouge")

# model = SentenceTransformer('all-MiniLM-L6-v2')
# model = SentenceTransformer('allenai/longformer-base-4096')
model = SentenceTransformer('all-mpnet-base-v1')
model.max_seq_length = 512
model.max_seq_length
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v1")

4.35.0
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def generate_consecutive_sentence_combinations(sentences):
    sentence_combinations = []

    # Add individual sentences
    sentence_combinations.extend(sentences)

    # Add combinations of consecutive sentences
    for window_size in range(2, len(sentences) + 1):
        for i in range(len(sentences) - window_size + 1):
            combined_sentences = ' '.join(sentences[i:i + window_size])
            sentence_combinations.append(combined_sentences)

    return sentence_combinations

In [4]:
def generate_consecutive_sentence_combinations(sentences):
    sentence_combinations = []

    # Add individual sentences
    sentence_combinations.extend(sentences)

    # Add combinations of consecutive sentences
    for window_size in range(2, len(sentences) + 1):
        for i in range(len(sentences) - window_size + 1):
            combined_sentences = ' '.join(sentences[i:i + window_size])
            # Tokenize and check if the token count exceeds model.max_seq_length
            token_count = len(tokenizer.tokenize(combined_sentences))
            if token_count <= model.max_seq_length:
                sentence_combinations.append(combined_sentences)

    return sentence_combinations

In [5]:
def compute_embeddings(sentences):
    return model.encode(sentences, convert_to_tensor=True)

In [6]:
laws_path = '/kaggle/input/laws-and-abstracts/laws.csv'
abstracts_path = '/kaggle/input/laws-and-abstracts/abstracts.csv'

In [7]:
laws_df = pd.read_csv(laws_path)
laws_df.head(20)

,law_index,section_index,para_index,text
0,0,0,0,Article 1. Definitions
1,0,0,1,The period of the anti-terrorist operation is the time between the date of entry into force of the Decree of the President of Ukraine “On the Decision of the National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014 and the date of enactment of the Decree of the President of Ukraine on the completion of the anti-terrorist operation or military actions within the territory of Ukraine.
2,0,0,2,"The territory of the anti-terrorist operation in the territory of Ukraine, where settlements defined in the list approved by the Cabinet of Ministers of Ukraine are located and where the anti-terrorist operation was conducted, which was initiated under the Decree of the President of Ukraine “On the Decision of National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014."
3,0,1,0,Article 2. The moratorium on the fulfilment of contractual obligations and the accrual of penalties and fines on the principal amount of debt under the credit and other contractual obligations
4,0,1,1,"For the duration of the anti-terrorist operation, it is prohibited to charge the citizens of Ukraine who are registered and permanently reside in or resettled in the period starting from 14 April 2014 from localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted, penalties and/or fines on the principal amount of debt under credit agreements and loan agreements dated from 14 April 2014. The same shall apply to legal entities and individual entrepreneurs who conduct (conducted) their economic activities within the territory of localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted."
5,0,1,2,"Banks and other financial institutions, as well as creditors, are obliged to cancel the penalties and/or fines accrued on the principal amount of debt under credit agreements and loan agreements during the period of the anti-terrorist operation for persons specified in this Article."
6,0,1,3,"Provisions of parts one and two of this Article shall not apply to the accrual of penalties and/or fines on the principal amount of debt under credit agreements and loan agreements concluded with legal entities and individual entrepreneurs located within the territory of anti-terrorist operation, except for localities from the lists provided for in part four of Article 4 of this Law, if such agreements are concluded after 1 January 2018 or to which, by agreement of the parties, changes were made after 1 January 2018 to extend the term for the fulfilment of obligations and/or reduce the amount of interest or punitive sanctions."
7,0,1,4,"For temporarily displaced higher education institutions and temporarily displaced scientific institutions to fulfil credit and other contractual obligations (except for agreements on the provision of educational services) that arose before moving to the territory controlled by the Ukrainian authorities in connection with the anti-terrorist operation, a moratorium is introduced for the period of the anti-terrorist operation."
8,0,1,5,"For the duration of the anti-terrorist operation, it is prohibited to charge penalties and/or fines on the principal amount of debt owed by temporarily displaced higher education institutions and temporarily displaced scientific institutions under credit and other agreements (except for agreements on the provision of educational services) that have not ceased, starting from the date of publication of the order to change their location by the central executive authority in the field of education and science."
9,0,1,6,"A temporari

In [8]:
abstracts_df = pd.read_csv(abstracts_path)
abstracts_df.head(20)

,abstract_index,para_index,text
0,0,0,Article 1 of the Law establishes the following:
1,0,1,"the anti-terrorist operation period is the time between the date of entry into force of the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014, and the date of entry into force of the Decree of the President of Ukraine on cessation of the anti-terrorist operation or military action on the territory of Ukraine;\nthe anti-terrorist operation territory is the territory of Ukraine containing population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was initiated according to the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014 was being carried out."
2,0,2,"During the anti-terrorist operation period, it is forbidden to accrue fines and/or penalties on the principal amounts of liabilities under credit agreements and loan agreements concluded after April 14, 2014 by citizens of Ukraine who are registered and permanently residing, or, after April 14, 2014, relocated from population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, and to legal entities and natural persons – entrepreneurs who conduct (conducted) their business activity on the territory of population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out (Article 2 of the Law)."
3,0,3,"According to Article 3 of the Law, bodies and officials that are authorized by law to carry out state supervision (oversight) in the sphere of commercial activity, during the anti-terrorist operation period and on the anti-terrorist operation territory are temporarily forbidden to hold scheduled and non-scheduled inspections of business entities that operate in the anti-terrorist operation area, except for unscheduled inspections of business entities that are classified as business entities with a high risk level, according to the business risk level assessment criteria approved by the Cabinet of Ministers of Ukraine."
4,0,4,Specifics of state registration of legal entities and natural persons – entrepreneurs from the anti-terrorist operation territory during the anti-terrorist operation period are determined by Article 4 of the Law.
5,0,5,Licenses and permits issued to business entities that operate on the anti-terrorist operation territory which have expired during the anti-terrorist operation period are considered to be extended for the duration of the anti-terrorist operation period (Article 5 of the Law).
6,0,6,"According to Article 6 of the Law, during the anti-terrorist operation period, business entities that operate on the anti-terrorist operation territory are exempt from paying for the use of state- and municipal owned land plots."
7,0,7,"Article 7 of the Law suspends rent payment for the use of state- and municipal owned property by business entities that operate on the anti-terrorist operation territory, for the duration of the anti-terrorist operation period."
8,0,8,"A moratorium on accrual of fines for late payments for municipal and utilities services, accrued for citizens of Ukraine residing in population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, is established by Article 8 of the Law."
9,0,9,"According to Article 9 of the Law, during the effect of this Law, mortgaged immovable property located on the anti-terrorist operation territory and 

In [9]:
def split_into_sentences_with_indices(text, abstract_index):
    sentences = sent_tokenize(text)
    return [{'abstract_index': abstract_index, 'sentence_index': i, 'text': sentence} for i, sentence in enumerate(sentences)]

abstracts_sentences_level_dataset = []

for abstract_index, group in abstracts_df.groupby('abstract_index'):
    concatenated_text = ' '.join(group['text'])
    abstracts_sentences_level_dataset.extend(split_into_sentences_with_indices(concatenated_text, abstract_index))

abstracts_sentences_level_df = pd.DataFrame(abstracts_sentences_level_dataset)

abstracts_sentences_level_df.head(20)

,abstract_index,sentence_index,text
0,0,0,"Article 1 of the Law establishes the following: the anti-terrorist operation period is the time between the date of entry into force of the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014, and the date of entry into force of the Decree of the President of Ukraine on cessation of the anti-terrorist operation or military action on the territory of Ukraine;\nthe anti-terrorist operation territory is the territory of Ukraine containing population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was initiated according to the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014 was being carried out."
1,0,1,"During the anti-terrorist operation period, it is forbidden to accrue fines and/or penalties on the principal amounts of liabilities under credit agreements and loan agreements concluded after April 14, 2014 by citizens of Ukraine who are registered and permanently residing, or, after April 14, 2014, relocated from population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, and to legal entities and natural persons – entrepreneurs who conduct (conducted) their business activity on the territory of population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out (Article 2 of the Law)."
2,0,2,"According to Article 3 of the Law, bodies and officials that are authorized by law to carry out state supervision (oversight) in the sphere of commercial activity, during the anti-terrorist operation period and on the anti-terrorist operation territory are temporarily forbidden to hold scheduled and non-scheduled inspections of business entities that operate in the anti-terrorist operation area, except for unscheduled inspections of business entities that are classified as business entities with a high risk level, according to the business risk level assessment criteria approved by the Cabinet of Ministers of Ukraine."
3,0,3,Specifics of state registration of legal entities and natural persons – entrepreneurs from the anti-terrorist operation territory during the anti-terrorist operation period are determined by Article 4 of the Law.
4,0,4,Licenses and permits issued to business entities that operate on the anti-terrorist operation territory which have expired during the anti-terrorist operation period are considered to be extended for the duration of the anti-terrorist operation period (Article 5 of the Law).
5,0,5,"According to Article 6 of the Law, during the anti-terrorist operation period, business entities that operate on the anti-terrorist operation territory are exempt from paying for the use of state- and municipal owned land plots."
6,0,6,"Article 7 of the Law suspends rent payment for the use of state- and municipal owned property by business entities that operate on the anti-terrorist operation territory, for the duration of the anti-terrorist operation period."
7,0,7,"A moratorium on accrual of fines for late payments for municipal and utilities services, accrued for citizens of Ukraine residing in population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, is established by Article 8 of the Law."
8,0,8,"According to Article 9 of the Law, during the effect of this Law, mortgaged immovable property located on the anti-terrorist operation territory and ow

In [10]:
laws_df_2_alg = laws_df.copy()
laws_df_2_alg

,law_index,section_index,para_index,text
0,0,0,0,Article 1. Definitions
1,0,0,1,The period of the anti-terrorist operation is the time between the date of entry into force of the Decree of the President of Ukraine “On the Decision of the National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014 and the date of enactment of the Decree of the President of Ukraine on the completion of the anti-terrorist operation or military actions within the territory of Ukraine.
2,0,0,2,"The territory of the anti-terrorist operation in the territory of Ukraine, where settlements defined in the list approved by the Cabinet of Ministers of Ukraine are located and where the anti-terrorist operation was conducted, which was initiated under the Decree of the President of Ukraine “On the Decision of National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014."
3,0,1,0,Article 2. The moratorium on the fulfilment of contractual obligations and the accrual of penalties and fines on the principal amount of debt under the credit and other contractual obligations
4,0,1,1,"For the duration of the anti-terrorist operation, it is prohibited to charge the citizens of Ukraine who are registered and permanently reside in or resettled in the period starting from 14 April 2014 from localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted, penalties and/or fines on the principal amount of debt under credit agreements and loan agreements dated from 14 April 2014. The same shall apply to legal entities and individual entrepreneurs who conduct (conducted) their economic activities within the territory of localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted."
...,...,...,...,...
58242,101,278,18,Execution of the resolution of the central executive authority implementing the state policy on the supervision and control over the compliance with labour legislation shall be entrusted to the bodies of the state executive service.
58243,101,278,19,Payment of the fine shall not exempt from the elimination of violations of labour legislation.
58244,101,278,20,Section XIX
58245,101,278,21,FINAL PROVISIONS


In [11]:
abstracts_sentences_level_df_2_alg = abstracts_sentences_level_df.copy()
abstracts_sentences_level_df_2_alg

,abstract_index,sentence_index,text
0,0,0,"Article 1 of the Law establishes the following: the anti-terrorist operation period is the time between the date of entry into force of the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014, and the date of entry into force of the Decree of the President of Ukraine on cessation of the anti-terrorist operation or military action on the territory of Ukraine;\nthe anti-terrorist operation territory is the territory of Ukraine containing population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was initiated according to the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014 was being carried out."
1,0,1,"During the anti-terrorist operation period, it is forbidden to accrue fines and/or penalties on the principal amounts of liabilities under credit agreements and loan agreements concluded after April 14, 2014 by citizens of Ukraine who are registered and permanently residing, or, after April 14, 2014, relocated from population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, and to legal entities and natural persons – entrepreneurs who conduct (conducted) their business activity on the territory of population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out (Article 2 of the Law)."
2,0,2,"According to Article 3 of the Law, bodies and officials that are authorized by law to carry out state supervision (oversight) in the sphere of commercial activity, during the anti-terrorist operation period and on the anti-terrorist operation territory are temporarily forbidden to hold scheduled and non-scheduled inspections of business entities that operate in the anti-terrorist operation area, except for unscheduled inspections of business entities that are classified as business entities with a high risk level, according to the business risk level assessment criteria approved by the Cabinet of Ministers of Ukraine."
3,0,3,Specifics of state registration of legal entities and natural persons – entrepreneurs from the anti-terrorist operation territory during the anti-terrorist operation period are determined by Article 4 of the Law.
4,0,4,Licenses and permits issued to business entities that operate on the anti-terrorist operation territory which have expired during the anti-terrorist operation period are considered to be extended for the duration of the anti-terrorist operation period (Article 5 of the Law).
...,...,...,...
2870,101,45,"State supervision over adherence to the labor legislation at enterprises, in institutions and organizations is performed by the central bodies of executive power on: supervision over labor protection; nuclear and radiation safety; fire supervision; sanitary epidemiological safety."
2871,101,46,The highest supervision over adherence to and accurate application of the laws on labor is performed by the General Prosecutor of Ukraine and the prosecutors subordinated thereof.
2872,101,47,Local state administrations and bodies of local self-government are entitled to fulfill respective authorities in the field of labor protection.
2873,101,48,The right to conduct public control over adherence to the labor legislation also have the trade unions and their associations.


In [12]:
# Assuming you want to retrieve the law and summary for law_index = 0
selected_law_index = 0

# Filter the laws dataframe for the selected law
first_law_df = laws_df_2_alg[laws_df_2_alg['law_index'] == selected_law_index]

# Since abstracts_sentences_level_df may not have a law_index, you need to rely on abstract_index
# Assuming that abstract_index corresponds to law_index (you need to adjust this if it's different)
first_abstract_df = abstracts_sentences_level_df_2_alg[abstracts_sentences_level_df_2_alg['abstract_index'] == selected_law_index]

In [13]:
first_law_df

,law_index,section_index,para_index,text
0,0,0,0,Article 1. Definitions
1,0,0,1,The period of the anti-terrorist operation is the time between the date of entry into force of the Decree of the President of Ukraine “On the Decision of the National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014 and the date of enactment of the Decree of the President of Ukraine on the completion of the anti-terrorist operation or military actions within the territory of Ukraine.
2,0,0,2,"The territory of the anti-terrorist operation in the territory of Ukraine, where settlements defined in the list approved by the Cabinet of Ministers of Ukraine are located and where the anti-terrorist operation was conducted, which was initiated under the Decree of the President of Ukraine “On the Decision of National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014."
3,0,1,0,Article 2. The moratorium on the fulfilment of contractual obligations and the accrual of penalties and fines on the principal amount of debt under the credit and other contractual obligations
4,0,1,1,"For the duration of the anti-terrorist operation, it is prohibited to charge the citizens of Ukraine who are registered and permanently reside in or resettled in the period starting from 14 April 2014 from localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted, penalties and/or fines on the principal amount of debt under credit agreements and loan agreements dated from 14 April 2014. The same shall apply to legal entities and individual entrepreneurs who conduct (conducted) their economic activities within the territory of localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted."
...,...,...,...,...
74,0,15,14,ensure that ministries and central executive authorities of Ukraine develop and review their regulatory acts implied in this Law.
75,0,15,15,"6. Within ten day from the enactment of this Law, the National Bank of Ukraine shall:"
76,0,15,16,bring their regulatory acts in compliance with Article 2 of this Law;
77,0,15,17,"give appropriate orders to all banks in the territory of Ukraine, regardless of the form of ownership, to ensure compliance with Article 2 of this Law."


In [14]:
first_abstract_df

,abstract_index,sentence_index,text
0,0,0,"Article 1 of the Law establishes the following: the anti-terrorist operation period is the time between the date of entry into force of the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014, and the date of entry into force of the Decree of the President of Ukraine on cessation of the anti-terrorist operation or military action on the territory of Ukraine;\nthe anti-terrorist operation territory is the territory of Ukraine containing population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was initiated according to the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014 was being carried out."
1,0,1,"During the anti-terrorist operation period, it is forbidden to accrue fines and/or penalties on the principal amounts of liabilities under credit agreements and loan agreements concluded after April 14, 2014 by citizens of Ukraine who are registered and permanently residing, or, after April 14, 2014, relocated from population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, and to legal entities and natural persons – entrepreneurs who conduct (conducted) their business activity on the territory of population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out (Article 2 of the Law)."
2,0,2,"According to Article 3 of the Law, bodies and officials that are authorized by law to carry out state supervision (oversight) in the sphere of commercial activity, during the anti-terrorist operation period and on the anti-terrorist operation territory are temporarily forbidden to hold scheduled and non-scheduled inspections of business entities that operate in the anti-terrorist operation area, except for unscheduled inspections of business entities that are classified as business entities with a high risk level, according to the business risk level assessment criteria approved by the Cabinet of Ministers of Ukraine."
3,0,3,Specifics of state registration of legal entities and natural persons – entrepreneurs from the anti-terrorist operation territory during the anti-terrorist operation period are determined by Article 4 of the Law.
4,0,4,Licenses and permits issued to business entities that operate on the anti-terrorist operation territory which have expired during the anti-terrorist operation period are considered to be extended for the duration of the anti-terrorist operation period (Article 5 of the Law).
5,0,5,"According to Article 6 of the Law, during the anti-terrorist operation period, business entities that operate on the anti-terrorist operation territory are exempt from paying for the use of state- and municipal owned land plots."
6,0,6,"Article 7 of the Law suspends rent payment for the use of state- and municipal owned property by business entities that operate on the anti-terrorist operation territory, for the duration of the anti-terrorist operation period."
7,0,7,"A moratorium on accrual of fines for late payments for municipal and utilities services, accrued for citizens of Ukraine residing in population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, is established by Article 8 of the Law."
8,0,8,"According to Article 9 of the Law, during the effect of this Law, mortgaged immovable property located on the anti-terrorist operation territory and ow

In [15]:
def split_to_sentences(row):
    doc = nlp(row["text"])
    sentences = [sent.text.strip() for sent in doc.sents]
    return pd.DataFrame({
        "law_index": [row["law_index"]] * len(sentences),
        "section_index": [row["section_index"]] * len(sentences),
        "para_index": [row["para_index"]] * len(sentences),
        "text": [row["text"]] * len(sentences),
        "sentences": sentences
    })

# Apply the function
first_law_with_sentences_df = pd.concat([split_to_sentences(row) for index, row in first_law_df.iterrows()]).reset_index(drop=True)
first_law_with_sentences_df

,law_index,section_index,para_index,text,sentences
0,0,0,0,Article 1. Definitions,Article 1.
1,0,0,0,Article 1. Definitions,Definitions
2,0,0,1,The period of the anti-terrorist operation is the time between the date of entry into force of the Decree of the President of Ukraine “On the Decision of the National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014 and the date of enactment of the Decree of the President of Ukraine on the completion of the anti-terrorist operation or military actions within the territory of Ukraine.,The period of the anti-terrorist operation is the time between the date of entry into force of the Decree of the President of Ukraine “On the Decision of the National Security and Defense Council of Ukraine of 13 April 2014
3,0,0,1,The period of the anti-terrorist operation is the time between the date of entry into force of the Decree of the President of Ukraine “On the Decision of the National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014 and the date of enactment of the Decree of the President of Ukraine on the completion of the anti-terrorist operation or military actions within the territory of Ukraine.,“On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014
4,0,0,1,The period of the anti-terrorist operation is the time between the date of entry into force of the Decree of the President of Ukraine “On the Decision of the National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014 and the date of enactment of the Decree of the President of Ukraine on the completion of the anti-terrorist operation or military actions within the territory of Ukraine.,No. 405/2014 and the date of enactment of the Decree of the President of Ukraine on the completion of the anti-terrorist operation or military actions within the territory of Ukraine.
...,...,...,...,...,...
105,0,15,15,"6. Within ten day from the enactment of this Law, the National Bank of Ukraine shall:",6.
106,0,15,15,"6. Within ten day from the enactment of this Law, the National Bank of Ukraine shall:","Within ten day from the enactment of this Law, the National Bank of Ukraine shall:"
107,0,15,16,bring their regulatory acts in compliance with Article 2 of this Law;,bring their regulatory acts in compliance with Article 2 of this Law;
108,0,15,17,"give appropriate orders to all banks in the territory of Ukraine, regardless of the form of ownership, to ensure compliance with Article 2 of this Law.","give appropriate orders to all banks in the territory of Ukraine, regardless of the form of ownership, to ensure compliance with Article 2 of this Law."


In [ ]:
# law_sentences = first_law_with_sentences_df['sentences'].tolist()
# first_law_sentence_combinations = generate_consecutive_sentence_combinations(law_sentences)

# abstract_sentences = first_abstract_df['text'].tolist()

# law_embeddings = compute_embeddings(first_law_sentence_combinations)
# summary_embeddings = compute_embeddings(abstract_sentences)

In [ ]:
# pairs = []

# for i, summary_embedding in enumerate(summary_embeddings):
#     similarities = util.pytorch_cos_sim(summary_embedding, law_embeddings)

#     # Move tensor to CPU and convert to numpy array if it's on GPU
#     if similarities.is_cuda:
#         similarities = similarities.cpu()

#     similarities_np = similarities.numpy()

#     # Find the index of the best match
#     best_match_index = np.argmax(similarities_np, axis=1)[0]

#     pairs.append({
#         "law_index": first_abstract_df.loc[i, 'abstract_index'],  
#         "summary_sentence": abstract_sentences[i],
#         "law_section_automatic": first_law_sentence_combinations[best_match_index]
#     })
    
# automatically_generated_pairs_first_law_df = pd.DataFrame(pairs)
# automatically_generated_pairs_first_law_df

In [ ]:
# automatically_generated_pairs_first_law_df.to_csv('/kaggle/working/first_law_generated_by_my_method_automatically_pairs_df.csv', index=False)

In [ ]:
automatically_generated_pairs_first_law_df = pd.read_csv('/kaggle/working/first_law_generated_by_my_method_automatically_pairs_df.csv')

Ground truth

In [ ]:
paired_ground_truth_df = pd.read_csv('/kaggle/input/paired-ground-truth/paired_ground_truth.csv')
paired_ground_truth_df.rename(columns={'similarity_score': 'html_based_method_similarity_score', 'law_section': 'law_section_html_based'}, inplace=True)
paired_ground_truth_df

In [ ]:
# Assuming you want to retrieve the law for law_index = 0
selected_law_index = 0

# Filter the laws dataframe for the selected law
first_law_paired_ground_truth_df = paired_ground_truth_df[paired_ground_truth_df['law_index'] == selected_law_index]
first_law_paired_ground_truth_df

In [ ]:
merged_df = pd.merge(first_law_paired_ground_truth_df, 
                     automatically_generated_pairs_first_law_df, 
                     on=['law_index', 'summary_sentence'], 
                     how='left')

merged_df

In [ ]:
# Create a new DataFrame as a copy of merged_df
df_with_both_cosine_similarity_metrics = merged_df.copy()

In [ ]:
# Compute embeddings for summary sentences and law sections
summary_embeddings = model.encode(df_with_both_cosine_similarity_metrics['summary_sentence'].tolist(), convert_to_tensor=True)
law_section_embeddings = model.encode(df_with_both_cosine_similarity_metrics['law_section_automatic'].tolist(), convert_to_tensor=True)

# Calculate cosine similarity
cosine_scores = util.pytorch_cos_sim(summary_embeddings, law_section_embeddings)

# Extracting the similarity scores as a list
similarity_scores = cosine_scores.diagonal().tolist()

# Adding the similarity scores to the DataFrame
df_with_both_cosine_similarity_metrics['automatic_method_similarity_score'] = similarity_scores
df_with_both_cosine_similarity_metrics

In [ ]:
def calculate_rouge_average(row):
    scores = rouge.compute(predictions=[row['law_section_html_based']], references=[row['law_section_automatic']], rouge_types=["rouge1", "rouge2", "rougeL"])
    rouge1_f1 = scores['rouge1'].mid.fmeasure
    rouge2_f1 = scores['rouge2'].mid.fmeasure
    rougeL_f1 = scores['rougeL'].mid.fmeasure
    return (rouge1_f1 + rouge2_f1 + rougeL_f1) / 3

In [ ]:
df_with_rouge_between_html_based_and_auto_generated = df_with_both_cosine_similarity_metrics.copy()
df_with_rouge_between_html_based_and_auto_generated['rouge_score'] = df_with_rouge_between_html_based_and_auto_generated.apply(calculate_rouge_average, axis=1)
df_with_rouge_between_html_based_and_auto_generated

Working on the whole laws/abstracts datasets

In [ ]:
# Apply the function to each row of the laws dataframe
all_laws_with_sentences = pd.concat([split_to_sentences(row) for index, row in laws_df_2_alg.iterrows()]).reset_index(drop=True)
all_laws_with_sentences

In [ ]:
# all_laws_with_sentences.to_csv('/kaggle/working/all_laws_with_sentences.csv', index=False)

In [ ]:
all_laws_with_sentences = pd.read_csv('/kaggle/working/all_laws_with_sentences.csv')

In [ ]:
# Assuming laws_df is your DataFrame containing the laws
# Group by 'law_index' and count the number of 'para_index' for each law
law_para_count = laws_df.groupby('law_index').count()['para_index'].reset_index()

# Sort by the count and get the top 50 laws with the smallest amount of 'para_index'
top_50_laws = law_para_count.sort_values(by='para_index').head(70)['law_index']

# Filter all_laws_with_sentences to include only rows from these top 50 laws
# filtered_laws_with_sentences = all_laws_with_sentences[all_laws_with_sentences['law_index'].isin(top_50_laws)]
filtered_laws_with_sentences = all_laws_with_sentences.copy()

In [ ]:
# Load the existing data
existing_data = pd.read_csv('/kaggle/working/automatically_generated_pairs_all_laws_df.csv')

# Get unique law indices already processed
processed_law_indices = set(existing_data['law_index'].unique())

# Filter out already processed laws
remaining_law_indices = set(filtered_laws_with_sentences['law_index'].unique()) - processed_law_indices

# Function to process a single law and append results to the file
def process_law_and_append(law_index, all_laws_df, abstracts_df, output_filepath, model):
    current_law_df = all_laws_df[all_laws_df['law_index'] == law_index]
    law_sentences = current_law_df['sentences'].tolist()
    law_sentence_combinations = generate_consecutive_sentence_combinations(law_sentences)

    current_abstracts_df = abstracts_df[abstracts_df['abstract_index'] == law_index]
    abstract_sentences = current_abstracts_df['text'].tolist()

    # Compute embeddings for both law and abstract sentences
    law_embeddings = model.encode(law_sentence_combinations, convert_to_tensor=True)
    summary_embeddings = model.encode(abstract_sentences, convert_to_tensor=True)

    # Store results for the current law
    batch_results = []
    for i, summary_embedding in enumerate(summary_embeddings):
        similarities = util.pytorch_cos_sim(summary_embedding, law_embeddings)
        best_match_index = torch.argmax(similarities).item()
        batch_results.append({
            "law_index": law_index,
            "summary_sentence": abstract_sentences[i],
            "law_section_automatic": law_sentence_combinations[best_match_index]
        })

    # Append results of current law to the CSV file
    pd.DataFrame(batch_results).to_csv(output_filepath, mode='a', header=False, index=False)
    
# Output file path
output_filepath = '/kaggle/working/automatically_generated_pairs_all_laws_df.csv'

# If the file doesn't exist, initialize it with headers
if not os.path.exists(output_filepath):
    with open(output_filepath, 'w') as file:
        file.write("law_index,summary_sentence,law_section_automatic\n")

# Process each unprocessed law and append results
for law_index in tqdm(remaining_law_indices):
    process_law_and_append(law_index, filtered_laws_with_sentences, abstracts_sentences_level_df_2_alg, output_filepath, model)

In [ ]:
automatically_generated_pairs_all_laws_df = pd.read_csv('/kaggle/working/automatically_generated_pairs_all_laws_df.csv')
automatically_generated_pairs_all_laws_df.head(20)

In [ ]:
# Extract unique law indices from automatically_generated_pairs_all_laws_df
unique_law_indices = automatically_generated_pairs_all_laws_df['law_index'].unique()

# Filter paired_ground_truth_df to include only rows where law_index is in unique_law_indices
filtered_paired_ground_truth_df = paired_ground_truth_df[paired_ground_truth_df['law_index'].isin(unique_law_indices)]

# Create a new DataFrame
matched_with_auto_generated_paired_ground_truth_df = filtered_paired_ground_truth_df.copy()

In [ ]:
merged_all_laws_df = pd.merge(matched_with_auto_generated_paired_ground_truth_df, 
                     automatically_generated_pairs_all_laws_df, 
                     on=['law_index', 'summary_sentence'], 
                     how='left')

In [ ]:
df_with_both_cosine_similarity_metrics_all_laws = merged_all_laws_df.copy()

In [ ]:
# Compute embeddings for summary sentences and law sections
summary_embeddings_all_laws = model.encode(df_with_both_cosine_similarity_metrics_all_laws['summary_sentence'].tolist(), convert_to_tensor=True)
law_section_embeddings_all_laws = model.encode(df_with_both_cosine_similarity_metrics_all_laws['law_section_automatic'].tolist(), convert_to_tensor=True)

# Calculate cosine similarity
cosine_scores_all_laws = util.pytorch_cos_sim(summary_embeddings_all_laws, law_section_embeddings_all_laws)

# Extracting the similarity scores as a list
similarity_scores_all_laws = cosine_scores_all_laws.diagonal().tolist()

# Adding the similarity scores to the DataFrame
df_with_both_cosine_similarity_metrics_all_laws['automatic_method_similarity_score'] = similarity_scores_all_laws
df_with_both_cosine_similarity_metrics_all_laws

In [ ]:
df_with_rouge_between_html_based_and_auto_generated_all_laws = df_with_both_cosine_similarity_metrics_all_laws.copy()
df_with_rouge_between_html_based_and_auto_generated_all_laws['rouge_score'] = df_with_rouge_between_html_based_and_auto_generated_all_laws.apply(calculate_rouge_average, axis=1)
df_with_rouge_between_html_based_and_auto_generated_all_laws

In [ ]:
mean_rouge_score = df_with_rouge_between_html_based_and_auto_generated_all_laws['rouge_score'].mean()
mean_html_based_method_similarity_score = df_with_rouge_between_html_based_and_auto_generated_all_laws['html_based_method_similarity_score'].mean()
mean_automatic_method_similarity_score = df_with_rouge_between_html_based_and_auto_generated_all_laws['automatic_method_similarity_score'].mean()

print(f"Mean rouge score: {mean_rouge_score}")
print(f"Mean html based method similarity score: {mean_html_based_method_similarity_score}")
print(f"Mean automatic method similarity score: {mean_automatic_method_similarity_score}")